In [17]:
# Importing Libraries
import torchmetrics
import torch
import torch.nn as nn
import seaborn as sns
from tqdm.notebook import tqdm
import torch.nn.functional as F
import torch.utils.data as data
from torch.utils.data import TensorDataset
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
import pandas as pd
import numpy as  np
import math
from torchmetrics.classification import MultilabelPrecisionRecallCurve

df = pd.read_csv("../3 - Cleaning & Transforming Data/cleaned_dataset.csv", index_col=0)
df.head()

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,ADR_AIU,ADR_Agranulocytosis,ADR_AntiHGMCRmyopathy,ADR_FDE,ADR_Juandice,ADR_MPeruption,ADR_MPexanthema,ADR_SJSI,ADR_SJSII,ADR_TEN,...,Allele_DRB1*16:02,Allele_DRB3*01:01,Allele_DRB3*02:02,Allele_DRB4*01:01,Allele_DRB5*01:01,Allele_DRB5*02,Cohort ethnicity_Black,Cohort ethnicity_Caucasian,Cohort ethnicity_Mongol,Cohort ethnicity_Unknown
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [7]:
df.iloc[:,:11]

,ADR_AIU,ADR_Agranulocytosis,ADR_AntiHGMCRmyopathy,ADR_FDE,ADR_Juandice,ADR_MPeruption,ADR_MPexanthema,ADR_SJSI,ADR_SJSII,ADR_TEN,No_ADR
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
17983,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
17984,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
17985,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
17986,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [8]:
df.iloc[:,:11].sum()

ADR_AIU                    494.0
ADR_Agranulocytosis        257.0
ADR_AntiHGMCRmyopathy       20.0
ADR_FDE                      6.0
ADR_Juandice                14.0
ADR_MPeruption             563.0
ADR_MPexanthema            971.0
ADR_SJSI                  2311.0
ADR_SJSII                  343.0
ADR_TEN                   1740.0
No_ADR                   11269.0
dtype: float64

In [9]:
pd.read_csv("../3 - Cleaning & Transforming Data/cleaned_dataset.csv",index_col = 0, dtype=np.float32)

,ADR_AIU,ADR_Agranulocytosis,ADR_AntiHGMCRmyopathy,ADR_FDE,ADR_Juandice,ADR_MPeruption,ADR_MPexanthema,ADR_SJSI,ADR_SJSII,ADR_TEN,...,Allele_DRB1*16:02,Allele_DRB3*01:01,Allele_DRB3*02:02,Allele_DRB4*01:01,Allele_DRB5*01:01,Allele_DRB5*02,Cohort ethnicity_Black,Cohort ethnicity_Caucasian,Cohort ethnicity_Mongol,Cohort ethnicity_Unknown
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17983.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
17984.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
17985.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
17986.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [10]:
class MainDatasetLoader(Dataset):
    def __init__(self, csv):
        self.xy_pandas = pd.read_csv(csv, index_col = 0, dtype=np.float32)
        self.xy_np = self.xy_pandas.to_numpy()
        self.x = torch.tensor(self.xy_np[:, 11:], dtype=torch.float32)
        self.y = torch.tensor(self.xy_np[:,:11], dtype=torch.float32)
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.x, self.y, random_state=0, test_size = 0.33, shuffle = True, stratify = self.y)
        self.train_dataset = TensorDataset(self.X_train, self.y_train)
        self.test_dataset = TensorDataset(self.X_test, self.y_test)
        
    def datareport(self):
        print(f"X_train: {len(self.X_train)}")
        print(f"X_test: {len(self.X_test)}")
        print(f"y_train: {len(self.y_train)}")
        print(f"y_test:, {len(self.y_test)}")
        print(f"input dimension: {self.x.shape}")
        print(f"output dimension: {self.y.shape}")
        
    def __getitem__(self, index): #มันรับ argument เดียวอ่าาา ;-; https://stackoverflow.com/questions/1685389/possible-to-use-more-than-one-argument-on-getitem
        return self.x[index], self.y[index]
        
    def __len__(self): #ตรงนี้เหมือนกันน
        return len(self.xy_pandas)

In [11]:
dataset = MainDatasetLoader("../3 - Cleaning & Transforming Data/cleaned_dataset.csv")

In [12]:
#dataset = MainDataset("../3 - Cleaning & Transforming Data/cleaned_dataset.csv")
#torch.manual_seed(0)
#train_set_size = int(len(dataset) * 0.8)
#valid_set_size = len(dataset) - train_set_size
#train_data, test_data = data.random_split(dataset, [train_set_size, valid_set_size])
#dataloader_train = DataLoader(dataset=train_data, batch_size=4, shuffle=True, num_workers=0)
#dataloader_test = DataLoader(dataset=test_data, batch_size=4, shuffle=True, num_workers=0)

#train_data = ClassifierDataset(dataset.X_train, dataset.y_train)
# #test_data = ClassifierDataset(dataset.X_test, dataset.y_test)
# dataloader_X_train = DataLoader(dataset=dataset.X_train, batch_size=32, shuffle=True, num_workers=0)
# dataloader_X_test = DataLoader(dataset=dataset.X_train, batch_size=32, shuffle=True, num_workers=0)
#train_dataset = ClassifierDataset(torch.from_numpy(dataset.X_train).float(), torch.from_numpy(dataset.y_train).long())
#val_dataset = ClassifierDataset(torch.from_numpy(dataset.X_val).float(), torch.from_numpy(dataset.y_val).long())
#test_dataset = ClassifierDataset(torch.from_numpy(dataset.X_test).float(), torch.from_numpy(dataset.y_test).long())

In [13]:
dataset.datareport()

X_train: 12051
X_test: 5937
y_train: 12051
y_test:, 5937
input dimension: torch.Size([17988, 431])
output dimension: torch.Size([17988, 11])


In [14]:
train_loader = DataLoader(dataset.train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset.test_dataset, batch_size=32, shuffle=True)

In [15]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define Model
class MainModel(nn.Module):    
    def __init__(self, input_dim, output_dim):
        super(MainModel, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, output_dim)
            #ไม่ใส่ softmax!
        )

    def forward(self, x):
        x = self.flatten(x)
        x = self.linear_relu_stack(x)
        return x

model = MainModel(dataset.x.shape[1],dataset.y.shape[1]).to(device)

# Print the Model class
print(model)

Using cpu device
MainModel(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=431, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=256, bias=True)
    (5): ReLU()
    (6): Linear(in_features=256, out_features=128, bias=True)
    (7): ReLU()
    (8): Linear(in_features=128, out_features=11, bias=True)
  )
)


In [16]:
# def multi_acc(y_pred, y_test):
#     y_pred_softmax = torch.log_softmax(y_pred, dim = 0)
#     _, y_pred_tags = torch.max(y_pred_softmax, dim = 0)    
    
#     correct_pred = (y_pred_tags == y_test).float()
#     acc = correct_pred.sum() / len(correct_pred)
    
#     acc = torch.round(acc * 100)
    
#     return acc


# accuracy_stats = {
#     'train': [],
#     "val": []
# }
# loss_stats = {
#     'train': [],
#     "val": []
# }

In [17]:
# print("Begin training.")
# for e in tqdm(range(1, epoch+1)):
    
#     # TRAINING
#     train_epoch_loss = 0
#     train_epoch_acc = 0
#     model.train()
#     for X_train_batch, y_train_batch in dataloader_train:
#         X_train_batch, y_train_batch = X_train_batch.to(device), y_train_batch.to(device)
#         optimizer.zero_grad()
        
#         y_train_pred = model(X_train_batch)
        
#         train_loss = loss_fn(y_train_pred, y_train_batch)
#         train_acc = multi_acc(y_train_pred, y_train_batch)
        
#         train_loss.backward()
#         optimizer.step()
        
#         train_epoch_loss += train_loss.item()
#         train_epoch_acc += train_acc.item()
 
#         print(f"loss: {train_loss:>7f} acc: {train_acc:>7f}")
        
#     # VALIDATION    
#     with torch.no_grad():
        
#         val_epoch_loss = 0
#         val_epoch_acc = 0
        
#         model.eval()
#         for X_val_batch, y_val_batch in dataloader_test:
#             X_val_batch, y_val_batch = X_val_batch.to(device), y_val_batch.to(device)
            
#             y_val_pred = model(X_val_batch)
                        
#             val_loss = loss_fn(y_val_pred, y_val_batch)
#             val_acc = multi_acc(y_val_pred, y_val_batch)
            
#             val_epoch_loss += val_loss.item()
#             val_epoch_acc += val_acc.item()
#     loss_stats['train'].append(train_epoch_loss/len(dataloader_train))
#     #loss_stats['val'].append(val_epoch_loss/len(val_loader))
#     accuracy_stats['train'].append(train_epoch_acc/len(dataloader_train))
#     #accuracy_stats['val'].append(val_epoch_acc/len(val_loader))
                              
    
#     print(f'Epoch {e+0:03}: | Train Loss: {train_epoch_loss/len(dataloader_train):.5f} | Test Loss: {val_epoch_loss/len(dataloader_test):.5f} | Train Acc: {train_epoch_acc/len(dataloader_train):.3f}| Test Acc: {val_epoch_acc/len(dataloader_test):.3f}')

In [18]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        #X = X.to(dtype=torch.float64)
        #y = y.to(dtype=torch.float64)

        #Write Tensorboard Graph Model
        #writer = SummaryWriter()
        #writer.add_graph(model)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}, [{current:>5d}/{size:>5d}]")




#ยังไม่แตะ!!
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [19]:
dataset.X_train.size()

torch.Size([12051, 431])

In [20]:
dataset.y_train.size()

torch.Size([12051, 11])

In [21]:
print(dataset.train_dataset.tensors[0].size())
print(dataset.train_dataset.tensors[1].size())

torch.Size([12051, 431])
torch.Size([12051, 11])


In [22]:
dataset.train_dataset[0]

(tensor([ 1.2740e+01, -1.5324e+00,  1.2740e+01,  3.5759e-01,  7.4836e-01,
          2.3627e+02,  2.2418e+02,  2.3609e+02,  8.8000e+01,  0.0000e+00,
          3.2342e-01, -3.5088e-01,  3.5088e-01,  3.2342e-01,  1.0000e+00,
          1.5000e+00,  2.0000e+00,  1.1260e+01, -1.4039e-01,  4.6659e-01,
         -2.3402e+00,  5.8473e-01, -2.5568e+00,  2.6791e+00, -7.1788e-01,
          2.6899e+00,  1.0803e+03,  2.2815e+01,  2.0803e+01,  8.8027e+00,
          1.3839e+01,  1.0743e+01,  4.8486e+00,  3.3152e+00,  3.3152e+00,
          2.2864e+00,  2.2864e+00,  1.5562e+00,  1.5562e+00, -2.5500e+00,
          2.6943e+06,  3.7196e+00,  4.0359e+00,  1.5692e+00,  1.2212e+02,
          5.7197e+00,  0.0000e+00,  2.8237e+00,  0.0000e+00,  0.0000e+00,
          6.0311e+00,  4.8999e+00,  4.7945e+00,  0.0000e+00,  0.0000e+00,
          4.8359e+01,  2.3212e+01,  0.0000e+00,  2.5082e+01,  2.1326e+01,
          2.9510e+01,  0.0000e+00,  0.0000e+00,  5.7197e+00,  0.0000e+00,
          4.8999e+00,  5.9466e+01,  0.

In [23]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
#optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
epochs = 5

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_loader, model, loss_fn, optimizer)
    #test(test_loader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 951761408.000000, [   32/12051]
loss:     nan, [ 3232/12051]
loss:     nan, [ 6432/12051]
loss:     nan, [ 9632/12051]
Epoch 2
-------------------------------
loss:     nan, [   32/12051]
loss:     nan, [ 3232/12051]
loss:     nan, [ 6432/12051]
loss:     nan, [ 9632/12051]
Epoch 3
-------------------------------
loss:     nan, [   32/12051]
loss:     nan, [ 3232/12051]
loss:     nan, [ 6432/12051]
loss:     nan, [ 9632/12051]
Epoch 4
-------------------------------
loss:     nan, [   32/12051]
loss:     nan, [ 3232/12051]
loss:     nan, [ 6432/12051]
loss:     nan, [ 9632/12051]
Epoch 5
-------------------------------
loss:     nan, [   32/12051]
loss:     nan, [ 3232/12051]
loss:     nan, [ 6432/12051]
loss:     nan, [ 9632/12051]
Done!


In [24]:
for batch, (X, y) in enumerate(train_loader):
    print(batch)
    print(X)
    print(y)
    break

0
tensor([[13.1569, -3.0975, 13.1569,  ...,  0.0000,  1.0000,  0.0000],
        [12.7395, -1.5324, 12.7395,  ...,  0.0000,  1.0000,  0.0000],
        [13.1345, -3.2474, 13.1345,  ...,  0.0000,  1.0000,  0.0000],
        ...,
        [ 8.7022, -3.8988,  8.7022,  ...,  1.0000,  0.0000,  0.0000],
        [12.6898, -5.1776, 12.6898,  ...,  0.0000,  1.0000,  0.0000],
        [12.7395, -1.5324, 12.7395,  ...,  0.0000,  1.0000,  0.0000]])
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0.,

In [25]:
for batch, (X, y) in enumerate(train_loader):
    X, y = X.to(device), y.to(device)
    X = y.to(dtype=torch.float64)
    y = y.to(dtype=torch.float64)
    break

In [13]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np


In [74]:
# show df
df = pd.read_csv("../2 - HLA Proteins/gene_A_aligned.csv")
df.dtypes
print(df)


             Alleles                                           Sequence   Char
0      A*01:01:01:01  MAVMAPRTLLLL-----LSG--ALALTQTWAGS--HSMRYFFTSV-...  598.0
1     A*01:01:01:02N  MAVMAPRTLLLL-----LSG--ALALTQTWAGS--HSMRYFFTSV-...  598.0
2      A*01:01:01:03  MAVMAPRTLLLL-----LSG--ALALTQTWAGS--HSMRYFFTSV-...  598.0
3      A*01:01:01:04  MAVMAPRTLLLL-----LSG--ALALTQTWAGS--HSMRYFFTSV-...  598.0
4      A*01:01:01:05  MAVMAPRTLLLL-----LSG--ALALTQTWAGS--HSMRYFFTSV-...  598.0
...              ...                                                ...    ...
7707         A*80:05  MAVMPPRTLLLL-----LSG--ALALTQTWAGS--HSMRYFFTSV-...  598.0
7708         A*80:06  MAVMPPRTLLLL-----LSG--ALALTQTWAGS--HSMRYFFTSV-...  598.0
7709         A*80:07  MAVMPPRTLLLL-----LSG--ALALTQTWAGS--HSMRYFFTSV-...  598.0
7710        A*80:08N  MAVMPPRTLLLL-----LSG--ALALTQTWAGS--HSMRYFFTSV-...  598.0
7711        A*80:09N  --------------------------------S--HSMRYFFTSV-...  598.0

[7712 rows x 3 columns]


In [73]:

# test concept
df = pd.DataFrame(
    {
    'row': ['asdaff', 'qweqwe', 'zxczxc']
}, 
index=[1,2,3])


df = pd.concat([df.iloc[:, 0], df['row'].apply(list).apply(pd.Series)], axis=1)

df = df.T.reset_index(drop=True).T
df.columns = [f'seq{i}' for i in range(df.shape[1])]


print(df)

     seq0 seq1 seq2 seq3 seq4 seq5 seq6
1  asdaff    a    s    d    a    f    f
2  qweqwe    q    w    e    q    w    e
3  zxczxc    z    x    c    z    x    c


In [72]:

# test concept
df = pd.DataFrame(
    {
    'row': ['asdaff', 'qweqwe', 'zxczxc']
}, 
index=[1,2,3])


print(df)

      row
1  asdaff
2  qweqwe
3  zxczxc


In [79]:

df = pd.read_csv("../2 - HLA Proteins/gene_A_aligned.csv")

#select column
df_seq = df[['Sequence']]



#seperate alphabet
df = pd.concat([df.iloc[:, 0], df['Sequence'].apply(list).apply(pd.Series)], axis=1)
df = df.T.reset_index(drop=True).T
df.columns = [f'seq{i}' for i in range(df.shape[1])]

print(df)



                seq0 seq1 seq2 seq3 seq4 seq5 seq6 seq7 seq8 seq9  ... seq589  \
0      A*01:01:01:01    M    A    V    M    A    P    R    T    L  ...      V   
1     A*01:01:01:02N    M    A    V    M    A    P    R    T    L  ...      -   
2      A*01:01:01:03    M    A    V    M    A    P    R    T    L  ...      V   
3      A*01:01:01:04    M    A    V    M    A    P    R    T    L  ...      V   
4      A*01:01:01:05    M    A    V    M    A    P    R    T    L  ...      V   
...              ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...    ...   
7707         A*80:05    M    A    V    M    P    P    R    T    L  ...      V   
7708         A*80:06    M    A    V    M    P    P    R    T    L  ...      V   
7709         A*80:07    M    A    V    M    P    P    R    T    L  ...      V   
7710        A*80:08N    M    A    V    M    P    P    R    T    L  ...      -   
7711        A*80:09N    -    -    -    -    -    -    -    -    -  ...      -   

     seq590 seq591 seq592 s

In [152]:
ohe = OneHotEncoder()
#สร้าง array seq1-598
seq_1_598 = ["seq" + str(i) for i in range(1, 599)]
#ยัดลง one hot
ohe.fit_transform(df[seq_1_598]).toarray

#make one hot each column
pd.get_dummies(df[seq_1_598])


,seq1_-,seq1_M,seq2_-,seq2_A,seq2_D,seq2_S,seq2_T,seq2_V,seq3_-,seq3_I,...,seq595_R,seq595_V,seq595_X,seq596_-,seq596_W,seq596_X,seq597_-,seq597_D,seq598_-,seq598_X
0,0,1,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,1,0
1,0,1,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,1,0
2,0,1,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,1,0
3,0,1,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,1,0
4,0,1,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7707,0,1,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,1,0
7708,0,1,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,1,0
7709,0,1,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,1,0
7710,0,1,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,1,0


In [4]:
import string

alphabet = list(string.ascii_uppercase + '-')
numbers = [str(i) for i in range(1, 600)]

seq_comple = []
for num in numbers:
    for char in alphabet:
        seq_comple.append('seq'+ num +  '_' + char)
print(seq_comple)

['seq1_A', 'seq1_B', 'seq1_C', 'seq1_D', 'seq1_E', 'seq1_F', 'seq1_G', 'seq1_H', 'seq1_I', 'seq1_J', 'seq1_K', 'seq1_L', 'seq1_M', 'seq1_N', 'seq1_O', 'seq1_P', 'seq1_Q', 'seq1_R', 'seq1_S', 'seq1_T', 'seq1_U', 'seq1_V', 'seq1_W', 'seq1_X', 'seq1_Y', 'seq1_Z', 'seq1_-', 'seq2_A', 'seq2_B', 'seq2_C', 'seq2_D', 'seq2_E', 'seq2_F', 'seq2_G', 'seq2_H', 'seq2_I', 'seq2_J', 'seq2_K', 'seq2_L', 'seq2_M', 'seq2_N', 'seq2_O', 'seq2_P', 'seq2_Q', 'seq2_R', 'seq2_S', 'seq2_T', 'seq2_U', 'seq2_V', 'seq2_W', 'seq2_X', 'seq2_Y', 'seq2_Z', 'seq2_-', 'seq3_A', 'seq3_B', 'seq3_C', 'seq3_D', 'seq3_E', 'seq3_F', 'seq3_G', 'seq3_H', 'seq3_I', 'seq3_J', 'seq3_K', 'seq3_L', 'seq3_M', 'seq3_N', 'seq3_O', 'seq3_P', 'seq3_Q', 'seq3_R', 'seq3_S', 'seq3_T', 'seq3_U', 'seq3_V', 'seq3_W', 'seq3_X', 'seq3_Y', 'seq3_Z', 'seq3_-', 'seq4_A', 'seq4_B', 'seq4_C', 'seq4_D', 'seq4_E', 'seq4_F', 'seq4_G', 'seq4_H', 'seq4_I', 'seq4_J', 'seq4_K', 'seq4_L', 'seq4_M', 'seq4_N', 'seq4_O', 'seq4_P', 'seq4_Q', 'seq4_R', 'seq4_S',